In [2]:
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import networkx as nx
"""
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
"""
import warnings
warnings.filterwarnings('ignore')

# Reset display options to default
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')


In [3]:
df=pd.read_csv('train.csv',sep="\t")
display(df)

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,Step Duration (sec),Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)
0,9938,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C1,2005-09-09 12:23:34.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,33.0,33.0,NaN,1,0,0,1,NaN,NaN
1,9939,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C2,2005-09-09 12:24:07.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,15.0,15.0,NaN,1,0,0,1,NaN,NaN
2,9940,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C1,2005-09-09 12:24:22.0,2005-09-09 12:25:16.0,2005-09-09 12:25:40.0,2005-09-09 12:25:40.0,78.0,NaN,78.0,0,2,0,1,Define Variable,1
3,9941,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C2,2005-09-09 12:25:40.0,2005-09-09 12:25:51.0,2005-09-09 12:27:24.0,2005-09-09 12:27:24.0,104.0,NaN,104.0,0,4,9,1,"Using small numbers~~Write expression, positiv...",1~~1~~1
4,9942,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R4C1,2005-09-09 12:27:24.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,6.0,6.0,NaN,1,0,0,1,Entering a given,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232739,1078856,JP20cryBg8,"Unit CTA1_24, Section CTA1_24-1",NOV05,1,R5C1,2006-04-12 10:17:08.0,2006-04-12 10:17:17.0,2006-04-12 10:18:19.0,2006-04-12 10:18:19.0,71.0,NaN,71.0,0,2,5,1,"Entering a given~~Convert unit, mixed",21~~1
232740,1078857,JP20cryBg8,"Unit CTA1_24, Section CTA1_24-1",NOV05,1,R1C2,2006-04-12 10:18:19.0,2006-04-12 10:18:40.0,2006-04-12 10:18:40.0,2006-04-12 10:18:40.0,21.0,21.0,NaN,1,0,0,1,NaN,NaN
232741,1078858,JP20cryBg8,"Unit CTA1_24, Section CTA1_24-1",NOV05,1,R2C2,2006-04-12 10:18:40.0,2006-04-12 10:18:44.0,2006-04-12 10:18:44.0,2006-04-12 10:18:44.0,4.0,4.0,NaN,1,0,0,1,Identifying units,7
232742,1078859,JP20cryBg8,"Unit CTA1_24, Section CTA1_24-1",NOV05,1,R3C2,2006-04-12 10:18:44.0,2006-04-12 10:18:58.0,2006-04-12 10:20:55.0,2006-04-12 10:20:55.0,131.0,NaN,131.0,0,1,0,1,"Using small numbers~~Write expression, positiv...",8~~5~~11


In [4]:
unique_problems=df['Problem Name'].unique() #PROBLEMAS TOTALES
len(unique_problems)

1021

In [5]:
unique_problems=df['Problem Hierarchy'].unique() #TEMAS
len(unique_problems)

138

In [6]:
unique_problems=df['Step Name'].unique() #STEPS DIFERENTES (se pueden solapar)
len(unique_problems)

60709

In [7]:
df[['Unit', 'Section']] = df['Problem Hierarchy'].str.split(', ', expand=True)
# Remove the words "Unit " and "Section " from the new columns
df['Unit'] = df['Unit'].str.replace('Unit ', '', regex=False)
df['Section'] = df['Section'].str.replace('Section ', '', regex=False)

In [8]:
display(df)

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,...,Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default),Unit,Section
0,9938,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C1,2005-09-09 12:23:34.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,...,33.0,NaN,1,0,0,1,NaN,NaN,CTA1_13,CTA1_13-1
1,9939,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C2,2005-09-09 12:24:07.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,...,15.0,NaN,1,0,0,1,NaN,NaN,CTA1_13,CTA1_13-1
2,9940,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C1,2005-09-09 12:24:22.0,2005-09-09 12:25:16.0,2005-09-09 12:25:40.0,2005-09-09 12:25:40.0,...,NaN,78.0,0,2,0,1,Define Variable,1,CTA1_13,CTA1_13-1
3,9941,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C2,2005-09-09 12:25:40.0,2005-09-09 12:25:51.0,2005-09-09 12:27:24.0,2005-09-09 12:27:24.0,...,NaN,104.0,0,4,9,1,"Using small numbers~~Write expression, positiv...",1~~1~~1,CTA1_13,CTA1_13-1
4,9942,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R4C1,2005-09-09 12:27:24.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,...,6.0,NaN,1,0,0,1,Entering a given,1,CTA1_13,CTA1_13-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232739,1078856,JP20cryBg8,"Unit CTA1_24, Section CTA1_24-1",NOV05,1,R5C1,2006-04-12 10:17:08.0,2006-04-12 10:17:17.0,2006-04-12 10:18:19.0,2006-04-12 10:18:19.0,...,NaN,71.0,0,2,5,1,"Entering a given~~Convert unit, mixed",21~~1,CTA1_24,CTA1_24-1
232740,1078857,JP20cryBg8,"Unit CTA1_24, Section CTA1_24-1",NOV05,1,R1C2,2006-04-12 10:18:19.0,2006-04-12 10:18:40.0,2006-04-12 10:18:40.0,2006-04-12 10:18:40.0,...,21.0,NaN,1,0,0,1,NaN,NaN,CTA1_24,CTA1_24-1
232741,1078858,JP20cryBg8,"Unit CTA1_24, Section CTA1_24-1",NOV05,1,R2C2,2006-04-12 10:18:40.0,2006-04-12 10:18:44.0,2006-04-12 10:18:44.0,2006-04-12 10:18:44.0,...,4.0,NaN,1,0,0,1,Identifying units,7,CTA1_24,CTA1_24-1
232742,1078859,JP20cryBg8,"Unit CTA1_24, Section CTA1_24-1",NOV05,1,R3C2,2006-04-12 10:18:44.0,2006-04-12 10:18:58.0,2006-04-12 10:20:55.0,2006-04-12 10:20:55.0,...,NaN,131.0,0,1,0,1,"Using small numbers~~Write expression, positiv...",8~~5~~11,CTA1_24,CTA1_24-1


In [9]:
def clean_skill(skill):
    # Patrón para buscar texto entre los primeros dos puntos y después de los primeros puntos y comas
    if str(skill)[0]=='[':
        if "SkillRule: done no solutions" in skill:
            return "done no solutions"
        else:
            skill=skill[skill.index(' ')+1:]
            skill=skill[:skill.index(';')]
            return skill
    
    else:
        return skill

In [10]:
df['KC(Default)']=df['KC(Default)'].apply(clean_skill)

In [11]:
df.columns

Index(['Row', 'Anon Student Id', 'Problem Hierarchy', 'Problem Name',
       'Problem View', 'Step Name', 'Step Start Time',
       'First Transaction Time', 'Correct Transaction Time', 'Step End Time',
       'Step Duration (sec)', 'Correct Step Duration (sec)',
       'Error Step Duration (sec)', 'Correct First Attempt', 'Incorrects',
       'Hints', 'Corrects', 'KC(Default)', 'Opportunity(Default)', 'Unit',
       'Section'],
      dtype='object')

In [12]:
#Reemplazar la columna problem Hierarchy por unit y section
dfa=df.drop(columns=['Problem Hierarchy'])
df=dfa.reindex(columns=['Row', 'Anon Student Id', 'Unit',
       'Section', 'Problem Name',
       'Problem View', 'Step Name', 'Step Start Time',
       'First Transaction Time', 'Correct Transaction Time', 'Step End Time',
       'Step Duration (sec)', 'Correct Step Duration (sec)',
       'Error Step Duration (sec)', 'Correct First Attempt', 'Incorrects',
       'Hints', 'Corrects', 'KC(Default)', 'Opportunity(Default)'])

In [13]:
df=df.drop(columns=['First Transaction Time','Correct Transaction Time','Correct Step Duration (sec)'])

In [14]:
"""pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)"""
df[df['Anon Student Id']=='JP20cryBg8']

,Row,Anon Student Id,Unit,Section,Problem Name,Problem View,Step Name,Step Start Time,Step End Time,Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)
232684,1078801,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R1C1,2006-03-22 09:46:41.0,2006-03-22 09:46:59.0,18.0,NaN,1,0,0,1,NaN,NaN
232685,1078802,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R2C1,2006-03-22 09:46:59.0,2006-03-22 09:47:05.0,6.0,NaN,1,0,0,1,Identifying units,1
232686,1078803,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R3C1,2006-03-22 09:47:05.0,2006-03-22 09:47:08.0,3.0,NaN,1,0,0,1,Define Variable,1
232687,1078804,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R1C2,2006-03-22 09:47:08.0,2006-03-22 09:47:19.0,11.0,NaN,1,0,0,1,NaN,NaN
232688,1078805,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R4C1,2006-03-22 09:47:19.0,2006-03-22 09:47:26.0,7.0,NaN,1,0,0,1,Entering a given,1
232689,1078806,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R5C1,2006-03-22 09:47:26.0,2006-03-22 09:47:32.0,6.0,NaN,1,0,0,1,"Entering a given~~Convert unit, standard",2~~1
232690,1078807,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R6C2,2006-03-22 09:47:32.0,2006-03-22 09:47:41.0,9.0,NaN,1,0,0,1,Entering a given,3
232691,1078808,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R3C2,2006-03-22 09:47:41.0,2006-03-22 09:48:27.0,46.0,46.0,0,0,2,1,Using simple numbers~~Using small numbers~~Wri...,1~~1~~1
232692,1078809,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R6C1,2006-03-22 09:48:27.0,2006-03-22 09:48:56.0,29.0,NaN,1,0,0,1,Using simple numbers~~Using small numbers~~Fin...,2~~2~~1
232693,1078810,JP20cryBg8,CTA1_24,CTA1_24-1,PROP16,1,R1C1,2006-04-12 09:56:01.0,2006-04-12 09:56:56.0,55.0,NaN,1,0,0,1,NaN,NaN


In [15]:
def get_problem_student_stats(df, unit, section, problem, alumno):
    df_p_alumno = df[(df['Unit'] == unit) & 
                     (df['Section'] == section) & 
                     (df['Problem Name'] == problem) & 
                     (df['Anon Student Id'] == alumno)]
     # Número de steps
    num_rows = df_p_alumno.shape[0]
    
    # Suma de fallos
    total_incorrects = df_p_alumno['Incorrects'].sum()
    
    # Suma de hints
    total_hints = df_p_alumno['Hints'].sum()
    
    # Tiempo total en segundos
    total_time_seconds = df_p_alumno['Step Duration (sec)'].sum()
    
    # Retornar los resultados
    return {
        'steps': num_rows,
        'total_incorrects': str(total_incorrects),
        'total_hints': str(total_hints),
        'total_steps': str(df_p_alumno['Corrects'].sum()+total_hints+total_incorrects),
        'total_time_seconds': str(total_time_seconds),
    }
   

In [ ]:
# Crear la estructura del diccionario
Units = list(df['Unit'].unique())
dicctotal = {}

for unit in Units:
    print(unit)
    df_unit = df[df['Unit'] == str(unit)]
    Sections = list(df_unit['Section'].unique())
    dicctotal[unit] = {}
    
    for section in Sections:
        df_section = df_unit[df_unit['Section'] == section]
        problems = list(df_section['Problem Name'].unique())
        dicctotal[unit][section] = {}
        
        for problem in problems:
            df_problem = df_section[df_section['Problem Name'] == problem]
            alumnos = list(df_problem['Anon Student Id'].unique())
            dicctotal[unit][section][problem] = {'Alumnos': {}, 'Stats': {}}
              # KC (skills) necesarias para el problema
            
            for alumno in alumnos:
                df_alumno = df_problem[df_problem['Anon Student Id'] == alumno]
                dicctotal[unit][section][problem]['Alumnos'][alumno] =get_problem_student_stats(df,unit,section,problem,alumno)
                
                    





In [ ]:
import json
with open('nested_data.json', 'w') as json_file:
    json.dump(dicctotal, json_file, indent=4)

In [ ]:
a = get_problem_student_stats(df, "CTA1_24", "CTA1_24-1", "L3REAL45", "JP20cryBg8")
display(a)

,Row,Anon Student Id,Unit,Section,Problem Name,Problem View,Step Name,Step Start Time,Step End Time,Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)
232684,1078801,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R1C1,2006-03-22 09:46:41.0,2006-03-22 09:46:59.0,18.0,NaN,1,0,0,1,NaN,NaN
232685,1078802,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R2C1,2006-03-22 09:46:59.0,2006-03-22 09:47:05.0,6.0,NaN,1,0,0,1,Identifying units,1
232686,1078803,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R3C1,2006-03-22 09:47:05.0,2006-03-22 09:47:08.0,3.0,NaN,1,0,0,1,Define Variable,1
232687,1078804,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R1C2,2006-03-22 09:47:08.0,2006-03-22 09:47:19.0,11.0,NaN,1,0,0,1,NaN,NaN
232688,1078805,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R4C1,2006-03-22 09:47:19.0,2006-03-22 09:47:26.0,7.0,NaN,1,0,0,1,Entering a given,1
232689,1078806,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R5C1,2006-03-22 09:47:26.0,2006-03-22 09:47:32.0,6.0,NaN,1,0,0,1,"Entering a given~~Convert unit, standard",2~~1
232690,1078807,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R6C2,2006-03-22 09:47:32.0,2006-03-22 09:47:41.0,9.0,NaN,1,0,0,1,Entering a given,3
232691,1078808,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R3C2,2006-03-22 09:47:41.0,2006-03-22 09:48:27.0,46.0,46.0,0,0,2,1,Using simple numbers~~Using small numbers~~Wri...,1~~1~~1
232692,1078809,JP20cryBg8,CTA1_24,CTA1_24-1,L3REAL45,1,R6C1,2006-03-22 09:48:27.0,2006-03-22 09:48:56.0,29.0,NaN,1,0,0,1,Using simple numbers~~Using small numbers~~Fin...,2~~2~~1


In [23]:
import json
with open('nested_data.json') as dicc2:
    try:
        # Read the file content
        content = dicc2.read()
        
        # Check if the file is empty
        if not content.strip():
            raise ValueError("File is empty")
        
        # Load the JSON content
        dicc_w_skills = json.loads(content)
        
        # Function to recursively iterate and print nested dictionaries
        def iterate_nested_dict(d, indent=0):
            for key, value in d.items():
                print(' ' * indent + str(key))
                if isinstance(value, dict):
                    iterate_nested_dict(value, indent + 4)
                else:
                    print(' ' * (indent + 4) + str(value))
        
        # Iterate through the top-level keys
        iterate_nested_dict(dicc_w_skills)

    except json.JSONDecodeError as e:
        print(f"JSONDecodeError: {e}")
    except ValueError as e:
        print(f"ValueError: {e}")

ValueError: File is empty
